In [2]:
import xarray as xr
from sklearn_xarray import wrap
from dask_ml.preprocessing import StandardScaler
import intake
from dask.distributed import Client
# client = Client(processes=False)
client = Client(threads_per_worker=4, n_workers=10, processes=False)
client

/home8/homedir3/perso/gmaze/miniconda3/envs/obidam36/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: inproc://134.246.146.54/23936/1 Dashboard: http://localhost:46469/status,Cluster Workers: 10 Cores: 40 Memory: 84.35 GB


In [3]:
catalog_url = 'https://raw.githubusercontent.com/obidam/m2poc2019/master/m2poc_data_catalog.yml'
ds = intake.Catalog(catalog_url).argo_global_homogeneous_sdl.read_chunked()
feat_0 = ds.TEMP.sel(DEPTH=0, method='nearest')
feat_1 = ds.TEMP.sel(DEPTH=-1000, method='nearest')
RES = xr.concat([feat_0, feat_1], dim='feature').chunk(chunks={'N_PROF':10000})
RES = RES.rename('ARGO')
# Remove obsolete attributes:
for a in RES.attrs.copy(): 
    del RES.attrs[a]
# Add feature names:
RES.attrs['feature_0'] = "%s at %0.2fm depth"%(feat_0.name,feat_0['DEPTH'])
RES.attrs['feature_1'] = "%s at %0.2fm depth"%(feat_1.name,feat_1['DEPTH'])
# We're good:
print(RES)

<xarray.DataArray 'ARGO' (feature: 2, N_PROF: 128910)>
dask.array<shape=(2, 128910), dtype=float32, chunksize=(1, 10000)>
Coordinates:
    LATITUDE   (N_PROF) float32 dask.array<shape=(128910,), chunksize=(10000,)>
    LONGITUDE  (N_PROF) float32 dask.array<shape=(128910,), chunksize=(10000,)>
    TIME       (N_PROF) datetime64[ns] dask.array<shape=(128910,), chunksize=(10000,)>
    DEPTH      (feature) float32 dask.array<shape=(2,), chunksize=(2,)>
Dimensions without coordinates: feature, N_PROF
Attributes:
    feature_0:  TEMP at 0.00m depth
    feature_1:  TEMP at -1000.00m depth


In [4]:
X = xr.DataArray(RES.values.T, dims=['sample','feature'])
print(X.values.shape)

(128910, 2)


In [5]:
# scaler = wrap(StandardScaler(), sample_dim='N_PROF')
scaler = wrap(StandardScaler(), sample_dim='sample')
%time scaler.fit(X)
print(scaler.estimator_.mean_)
print(scaler.estimator_.scale_)
scaler.transform(X)

CPU times: user 37 ms, sys: 1 ms, total: 38 ms
Wall time: 2.37 ms
[17.673868   4.2371645]
[9.406726  1.6141378]


<xarray.DataArray (sample: 128910, feature: 2)>
array([[-1.551811, -1.29166 ],
       [-1.82948 , -1.280055],
       [-1.588464, -1.29136 ],
       ...,
       [-1.656578, -0.882204],
       [-0.824594, -0.895902],
       [-1.067999, -0.82858 ]], dtype=float32)
Dimensions without coordinates: sample, feature

In [11]:
# Artificially increase the size of the dataset:
X = xr.concat([X] * 100, dim='sample')

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 6.23 GB -- Worker memory limit: 8.44 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 6.31 GB -- Worker memory limit: 8.44 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 6.34 GB -- Worker memory limit: 8.44 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 6.38 GB -- Worker memory limit: 8.44 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 6.43 GB -- Worker memory limit: 8.44 GB
distributed.worker - WARN

In [12]:
print(X)
print('This dataset holds: %.3f GB' % (X.nbytes / 1e9))

<xarray.DataArray (sample: 1289100000, feature: 2)>
array([[-1.551811, -1.29166 ],
       [-1.82948 , -1.280055],
       [-1.588464, -1.29136 ],
       ...,
       [-1.656578, -0.882204],
       [-0.824594, -0.895902],
       [-1.067999, -0.82858 ]], dtype=float32)
Dimensions without coordinates: sample, feature
This dataset holds: 10.313 GB


distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 10.56 GB -- Worker memory limit: 8.44 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 10.56 GB -- Worker memory limit: 8.44 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 10.56 GB -- Worker memory limit: 8.44 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 10.56 GB -- Worker memory limit: 8.44 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 10.56 GB -- Worker memory limit: 8.44 GB
distributed.worker -

In [10]:
scaler = wrap(StandardScaler(),sample_dim='sample')
%time scaler.fit(X)
print(scaler.estimator_.mean_)
print(scaler.estimator_.scale_)

CPU times: user 92 ms, sys: 24 ms, total: 116 ms
Wall time: 102 ms
[-9.222270e-08  2.371806e-07]
[0.99999946 0.9999995 ]
